In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Hannanum
import nltk
import gensim
import re

In [2]:
hannanum = Hannanum()

In [16]:
df = pd.read_csv("filelink.csv", encoding='euc-kr',header=None)

In [17]:
df.columns = ['user_id', 'timestamp', 'title', 'contents']

In [18]:
df.tail()

,user_id,timestamp,title,contents
45,4566847029,2018.08.20 11시 12분,전문적으로 부캐들 모아서 빠대학살하는 팟의 존재.,1:부캐가 금지된것도 아닌데 뭔 소리냐 싶을수도있는데.전체채팅으로 이렇게 광고를 대...
46,4566846995,2018.08.19 00시 57분,개인적으로 생각하는 현 빠대 패왕,"1:레이너, 아즈모단, 나지보.레이너랑 아즈는 언제 너프 더 쳐먹일지 궁금하고 나지..."
47,4566847015,2018.08.19 20시 16분,영웅리그 등급 실버1 200점에서 실버2 400점으로 내려갔습니다.,1:영웅리그 등급 실버1 200점에서 실버2 400점으로 내려갔습니다.영웅리그를 한...
48,4566837039,2018.08.19 13시 33분,진절머리나는 일방적인 매칭 시스템,1:아주 무능한 티를 팍팍 내는 운영이군요.단 한번도 대등한 게임이 없습니다. 단 ...
49,4566847014,2018.08.19 18시 44분,히오스 운영진은 탱녀입니다.,1:3탱 1힐 1아바로 매칭을 잡아주는 그대들은 탱녀입니다.


In [23]:
a = hannanum.morphs(df['contents'][1])

In [56]:
for i in range(len(df)):
    tokens = hannanum.morphs(df["contents"][i])
    df["token_contents"][i] = tokens
    

/Users/Yeoni/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [57]:
df.head()

,user_id,timestamp,title,contents,token_contents
0,4566837162,2018.08.24 14시 08분,아..매칭좀 제대로 일좀해라,"1:2인팀하고 솔로큐하고 4인,5인큐랑은 만나게하면 안된다고애초에 상대가 되냐? 조...","[1:2인팀하, 이, 고, 솔로큐, 하고, 4인,5인큐랑, 은, 만나, 게, 하, ..."
1,4566827161,2018.08.25 03시 29분,계급시스템을 승패 숫자로만 바꿔야함,1:1. 역전하려고 안함... 그냥 빠른포기... 해봤자 데스 늘어서 애들 그냥 성...,"[1, :, 1, ., 역전, 하, 려고, 안하, ㅁ, ..., 그냥, 빠른포, 이..."
2,4566837171,2018.08.25 01시 55분,겜하는데 자리비움하는애들 징벌좀 제대로해라,1:자리비움하는 애랑 나가는애 때문에 몇판을 지냐자리비움하는 애들이나 나가는애들 영...,"[1:자리비움하, 는, 애, 랑, 나가는애, 때문, 에, 몇판, 을, 지냐자리비움하..."
3,4566837166,2018.08.24 17시 01분,블리자드는 정신차려라,1:제발 새로운 캐릭터 나오면 빠대에서 캐릭터 하나가 다 해먹는게 말이되냐아무리 홍...,"[1:제발, 새롭, 은, 캐릭터, 나, 아, 오, 면, 빠대, 에서, 캐릭터, 하나..."
4,4566857118,2018.08.24 15시 21분,왜 자꾸 매칭시스템을 이상하게 바꾸는거에요,1:왜 자꾸 매칭시스템을 이상하게 바꾸는거에요이러다가 사용자들 얼마 안남게생겼네요앞...,"[1:왜, 자꾸, 매칭시스템, 을, 이상, 하, 게, 바꾸는거에요이러다, 가, 사용..."


In [58]:
df["token_contents"][0]

['1:2인팀하',
 '이',
 '고',
 '솔로큐',
 '하고',
 '4인,5인큐랑',
 '은',
 '만나',
 '게',
 '하',
 '면',
 '안된다고애초',
 '에',
 '상대',
 '가',
 '되',
 '냐',
 '?',
 '조합다맞춰서',
 '4',
 '이',
 'ㄴ',
 '5',
 '이',
 'ㄴ',
 '짜',
 '는',
 '애들하고솔',
 '로',
 '듀오돌리',
 '는',
 '애',
 '들',
 '빠대하',
 '이',
 '고',
 '조합',
 '부터가',
 '에러',
 '이',
 'ㄴ데',
 '게임자체',
 '가',
 '되',
 '냐',
 '고',
 '?',
 '4인큐',
 '이상',
 '은',
 '무조건',
 '4',
 '이',
 'ㄴ',
 '5인큐',
 '하고만',
 '만나',
 '게',
 '매칭해야지;;;,2:저',
 '도',
 '그냥',
 '접',
 '을라고',
 '생각중입니다,3:초갈',
 '길',
 'ㄹ',
 '바',
 '아우리',
 '엘',
 '개노답',
 '5인큐,4:잘하',
 '는',
 '팟',
 '도',
 '개',
 '문제',
 '이',
 'ㄴ데',
 '못하',
 '는',
 '애',
 '들',
 '끼리',
 '뭉치',
 'ㄴ',
 '팟',
 '이',
 '아군',
 '으로',
 '나타나',
 '아',
 '보',
 '아',
 '요',
 '블리자드',
 '에',
 '폭탄',
 '가방',
 '던지',
 '고',
 '싶',
 '어']

In [59]:
from collections import Counter

In [61]:
import random

def p_topic_given_document(topic, d, alpha=0.1):
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

def topic_weight(d, word, k):
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

def sample_from(weights):
    total = sum(weights)
    rnd = total * random.random()
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

In [67]:
documents = df["token_contents"][::1]

In [68]:
documents

0     [1:2인팀하, 이, 고, 솔로큐, 하고, 4인,5인큐랑, 은, 만나, 게, 하, ...
1     [1, :, 1, ., 역전, 하, 려고, 안하, ㅁ, ..., 그냥, 빠른포, 이...
2     [1:자리비움하, 는, 애, 랑, 나가는애, 때문, 에, 몇판, 을, 지냐자리비움하...
3     [1:제발, 새롭, 은, 캐릭터, 나, 아, 오, 면, 빠대, 에서, 캐릭터, 하나...
4     [1:왜, 자꾸, 매칭시스템, 을, 이상, 하, 게, 바꾸는거에요이러다, 가, 사용...
5     [1, :, 진짜, 욕나올정도, 로, 말도안돼게잡히는데, 어떻게생각하는거, ??, ...
6     [1:김민영정,, dansime, <-, 듀오, 였는데, 제재, 부탁드리, ㅂ니다,...
7     [1, :, 이, 빠대, 조합, 과, 관련해서, 얼마나, 오랜시간, 얼마나, 많, ...
8     [1, :, 사실, 알, 나, 트레, 많이한, 유저, 로써, 할말, 은, 아닌거같은...
9     [1:개같은빠대, 탱, 숫자, 는, 좀, 맞춰줘요개발자한테, 가긴, 가, ㅁ, ??...
10    [1, :, 영웅, 킬, 이나, 돌격병, 경험치, 먹, 는, 범위, 정도, 에, 속...
11    [1:2판연속, 5인팟걸릴거, 이, 면, 그, 5인팟끼리, 매칭시키는, 게, 맞, ...
12    [1, :, 우리팀, 은, 2인큐, 이, ㄴ데, 5인파티, 몇판, 째, 만나, 는,...
13    [1:1탱, 1지원, 3암살자임조합, 무슨생각, 으로, 던지, 냐, ?, 테섭, 이...
14    [1, :, 안녕, 하, 세, 요, 히어로스, 스톰, 관계, 이, 자, 여러문, 다...
15    [1, :, 시, 마다, 형제, 는, 너프, 가, 더, 없, 나, ..., 겐지, ...
16    [1:켈투자드, 버그, 있, 는, 듯, ?, 기본, 퀘스트, w, 중심, 적중, 하...
17    [1:신캐릭들같은경우, 보, 면, 시간, 이, 좀, 지나, 아ㄴ데, 도, 1

In [70]:
random.seed(0)
K=4
document_topics = [[random.randrange(K) for word in document]
                    for document in documents]
document_topic_counts = [Counter() for _ in documents]
topic_word_counts = [Counter() for _ in range(K)]
topic_counts = [0 for _ in range(K)]
document_lengths = [len(document) for document in documents]
distinct_words = set(word for document in documents for word in document)
V = len(distinct_words)
D = len(documents)

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

In [72]:
document_topic_counts[0]

Counter({0: 70, 1: 32, 2: 0, 3: 0})

In [73]:
topic_word_counts[0]

Counter({'game': 0,
         '이상하게잡히거': 1,
         'see': 0,
         '시공파편': 0,
         '1차': 0,
         '8월': 0,
         '욕': 0,
         'next': 0,
         '해': 2,
         '새롭': 4,
         '먹': 0,
         '감사': 0,
         '좋': 5,
         '갑자기': 2,
         '다하': 0,
         '명중시': 0,
         '시ㄴ': 1,
         '전혀안하고상자': 0,
         '마다': 0,
         '레오릭': 0,
         '니네': 1,
         '난무': 0,
         '스트레스': 0,
         '10': 2,
         '감동추가': 1,
         '티란데': 1,
         '왠만하': 1,
         'the': 0,
         '망령걸음': 0,
         '도타2던': 0,
         'Hio': 0,
         '즐기': 5,
         '잖아': 0,
         '16일': 0,
         '질': 0,
         '인공지능하는데': 0,
         '여전히': 0,
         '4분기': 0,
         '/': 2,
         '하긴': 1,
         '듯': 2,
         '골드': 0,
         'ranking': 0,
         '간극': 0,
         'MMR고정치': 0,
         '년.들': 1,
         '..': 9,
         '부담': 1,
         '얼마': 0,
         'hero': 0,
         '입장에사': 1,
         '■': 0,
         '행위▶혐오감':